# Lab | Making predictions with logistic regression

In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

### Instructions

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.
2. Create a query to get the list of films and a boolean indicating if it was rented last month (August 2005). This would be our target variable.
3. Read the data into a Pandas dataframe.
4. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict this variable from the cleaned data.
6. Evaluate the results.


In [3]:
# Import libraries
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sqlalchemy import create_engine
import getpass



In [22]:
pass_ = getpass.getpass('Please input your connection credentials:')
connection_string = 'mysql+pymysql://root:'+ pass_ + '@localhost/sakila'
sakila_engine = create_engine(connection_string)



In [30]:
# For this problem we will likely need data from the rental, inventory and film tables of the sakila database.
# Primarily, we need the  rental_id, rental_date, film_id and title data.
query = 'SELECT r.rental_id, r.rental_date, i.film_id, c.name, f.title, f.length, f.rental_rate, f.rating, f.rental_duration, f.release_year from sakila.rental r join sakila.inventory i using(inventory_id) join sakila.film f on i.film_id = f.film_id join sakila.film_category fc on f.film_id = fc.film_id join sakila.category c on(fc.category_id = c.category_id)where YEAR(convert(r.rental_date,DATE)) = 2005 and YEAR(convert(r.rental_date,DATE)) is not null group by r.rental_id;'
label_query = ('select j1.rental_id, j1.rental_date, j1.film_id,j1.title,(month(j1.rental_date)= 08) as August_rental from (SELECT r.rental_id, r.rental_date, i.film_id, c.name, f.title, f.length, f.rental_rate, f.rating, f.rental_duration, f.release_year from sakila.rental r join sakila.inventory i using(inventory_id) join sakila.film f on i.film_id = f.film_id join sakila.film_category fc on f.film_id = fc.film_id join sakila.category c on(fc.category_id = c.category_id)where YEAR(convert(r.rental_date,DATE)) = 2005 and YEAR(convert(r.rental_date,DATE)) is not null group by r.rental_id) as j1 group by j1.rental_id;') 
rentals = pd.read_sql_query(query, sakila_engine)
August_rentals = pd.read_sql_query(label_query, sakila_engine)
rentals.head()
#August_rentals.head()

,rental_id,rental_date,film_id,name,title,length,rental_rate,rating,rental_duration,release_year
0,10895,2005-08-02 01:16:59,19,Action,AMADEUS HOLY,113,0.99,PG,6,2006
1,12268,2005-08-18 04:26:54,19,Action,AMADEUS HOLY,113,0.99,PG,6,2006
2,3150,2005-06-20 20:35:28,19,Action,AMADEUS HOLY,113,0.99,PG,6,2006
3,5072,2005-07-09 05:01:58,19,Action,AMADEUS HOLY,113,0.99,PG,6,2006
4,9080,2005-07-30 08:02:39,19,Action,AMADEUS HOLY,113,0.99,PG,6,2006


In [31]:
# View the data types
rentals.dtypes
# View the shape of the data set
rentals.shape




(15862, 10)

In [32]:
# View the target data set. 
August_rentals.shape
#August_rentals.dtypes
#August_rentals.isna().sum()
August_rentals.head()

,rental_id,rental_date,film_id,title,August_rental
0,10895,2005-08-02 01:16:59,19,AMADEUS HOLY,1
1,12268,2005-08-18 04:26:54,19,AMADEUS HOLY,1
2,3150,2005-06-20 20:35:28,19,AMADEUS HOLY,0
3,5072,2005-07-09 05:01:58,19,AMADEUS HOLY,0
4,9080,2005-07-30 08:02:39,19,AMADEUS HOLY,0


In [33]:
# Review statistical information about the predictors
rentals.describe(include = 'all', datetime_is_numeric= True)


,rental_id,rental_date,film_id,name,title,length,rental_rate,rating,rental_duration,release_year
count,15862.000000,15862,15862.000000,15862,15862,15862.000000,15862.000000,15862,15862.000000,15862.0
unique,NaN,NaN,NaN,16,958,NaN,NaN,5,NaN,NaN
top,NaN,NaN,NaN,Sports,BUCKET BROTHERHOOD,NaN,NaN,PG-13,NaN,NaN
freq,NaN,NaN,NaN,1164,34,NaN,NaN,3543,NaN,NaN
mean,7960.041546,2005-07-20 23:24:23.714285824,501.256651,NaN,NaN,114.992750,2.943978,NaN,4.937208,2006.0
min,1.000000,2005-05-24 22:53:30,1.000000,NaN,NaN,46.000000,0.990000,NaN,3.000000,2006.0
25%,3968.250000,2005-07-06 22:47:21.500000,255.000000,NaN,NaN,81.000000,0.990000,NaN,4.000000,2006.0
50%,7934.500000,2005-07-28 12:33:13.500000,498.000000,NaN,NaN,114.000000,2.990000,NaN,5.000000,2006.0
75%,11920.750000,2005-08-17 16:11:55,753.000000,NaN,NaN,148.000000,4.990000,NaN,6.000000,2006.0
max,16049.000000,2005-08-23 22:50:12,1000.000000,NaN,NaN,185.000000,4.990000,NaN,7.000000,2006.0


In [111]:
rentals['Month'] = rentals['rental_date'].dt.strftime("%b")
rentals.head()



,rental_id,rental_date,film_id,name,title,length,rental_rate,rating,rental_duration,release_year,Month
0,10895,2005-08-02 01:16:59,19,Action,AMADEUS HOLY,113,0.99,PG,6,2006,Aug
1,12268,2005-08-18 04:26:54,19,Action,AMADEUS HOLY,113,0.99,PG,6,2006,Aug
2,3150,2005-06-20 20:35:28,19,Action,AMADEUS HOLY,113,0.99,PG,6,2006,Jun
3,5072,2005-07-09 05:01:58,19,Action,AMADEUS HOLY,113,0.99,PG,6,2006,Jul
4,9080,2005-07-30 08:02:39,19,Action,AMADEUS HOLY,113,0.99,PG,6,2006,Jul


In [174]:
# Create a column for the number of each film_id rented in each month 
films_month = rentals.groupby(['Month','film_id']).agg({'rental_id':'count'})
films_month = pd.DataFrame(films_month).reset_index()
films_month.rename(columns = {'rental_id':'rentals/month'}, inplace= True)
films_month
films_month['rentals/month'].unique()

# Find some way to apply each count to the respective film_id ... somehow.


array([ 9,  4,  6,  3,  7,  5, 11,  8,  2, 12, 10,  1, 13, 14, 15, 16],
      dtype=int64)

In [176]:
# split data into training and testing sets.
from sklearn.model_selection import train_test_split

# Remove non-essential columns from the training datasets.
# 'rental_id' offers no information beyond the index of the rental.
# 'rental_date' may show direct correlation with the target, which might affect the model's out of sample performance.
# 'title' has too many unique values to be useful as a categorical predictor.
# 'release_year' shows no variation across the distribution.
# 'Month' is directly correlated with the target.
X = rentals.drop(['rental_id','film_id', 'rental_date', 'title','release_year', 'Month'],axis = 1)
y = August_rentals['August_rental']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
# Review split data.
display(X_train.shape)
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)
X_train.head()

(11103, 5)

(4759, 5)

(11103,)

(4759,)

,name,length,rental_rate,rating,rental_duration
8700,Foreign,181,0.99,G,6
3858,Classics,98,4.99,PG-13,6
96,Action,56,0.99,G,7
14327,Sports,105,0.99,NC-17,3
3130,Children,84,2.99,PG,5


# Transform the data for modelling
1.
2. Split the training and testing data into numerical and categorical information for normalization.

In [177]:
# Transform data for model training and testing
X_train_nums = X_train._get_numeric_data()
X_test_nums = X_test._get_numeric_data()
X_train_cats = X_train.select_dtypes(include = 'object')
X_test_cats = X_test.select_dtypes(include = 'object')
X_train_cats

,name,rating
8700,Foreign,G
3858,Classics,PG-13
96,Action,G
14327,Sports,NC-17
3130,Children,PG
...,...,...
13418,Sci-Fi,NC-17
5390,Documentary,PG
860,Action,R
15795,Travel,R


In [178]:
from sklearn.preprocessing import StandardScaler
# define function to output normalized and standardized training and testing data.

def num_scaler(Xtraining, Xtesting):
    # Normalize the numerical data with MinMaxScaler
    normal_transform = MinMaxScaler().fit(Xtraining)
    # Normalize the numerical date with StandardScaler
    standard_transform = StandardScaler().fit(Xtraining)
    # Transform the training data
    Xtrain_normalized = pd.DataFrame(normal_transform.transform(Xtraining), columns = Xtraining.columns)
    Xtrain_standardized = pd.DataFrame(standard_transform.transform(Xtraining), columns = Xtraining.columns)
    # Transform the testing data
    Xtest_normalized = pd.DataFrame(normal_transform.transform(Xtesting), columns = Xtraining.columns)
    Xtest_standardized = pd.DataFrame(standard_transform.transform(Xtesting), columns = Xtraining.columns)
    return Xtrain_normalized, Xtrain_standardized, Xtest_normalized, Xtest_standardized

# Transform training and testing data.
X_train_norm, X_train_stand, X_test_norm, X_test_stand = num_scaler(X_train_nums,X_test_nums)


In [179]:
# Transform categorical data
from sklearn.preprocessing import OneHotEncoder

def cat_encoder(Xtraining, Xtesting):
    # create encoder object
    OH_encoder = OneHotEncoder().fit(Xtraining)
    #create header list
    category_columns = []
    for columns in Xtraining.columns:
        for column_header in Xtraining[columns].unique():
            category_columns.append(column_header)
    # encode training data and re-assign result as dataframe with column names
    Xtrain_encoded = OH_encoder.transform(Xtraining).toarray()        
    Xtrain_encoded = pd.DataFrame(Xtrain_encoded, columns = category_columns)
    # encode testing data and re-assign result as dataframe with column names
    Xtest_encoded = OH_encoder.transform(Xtesting).toarray()        
    Xtest_encoded = pd.DataFrame(Xtest_encoded, columns = category_columns)
    
    # return the encoded datas frames
    return Xtrain_encoded, Xtest_encoded

X_train_cats_encoded, X_test_cats_encoded = cat_encoder(X_train_cats, X_test_cats)
X_train_cats_encoded.shape
X_test_cats_encoded.shape
X_train_cats_encoded.head()





,Foreign,Classics,Action,Sports,Children,Animation,Sci-Fi,New,Documentary,Music,...,Horror,Games,Drama,Family,Comedy,G,PG-13,NC-17,PG,R
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [180]:
# Concatenate numerical and categorical data for training and testing.
X_train1 = pd.concat([X_train_norm,X_train_cats_encoded], axis = 1)
X_train2 = pd.concat([X_train_stand,X_train_cats_encoded], axis = 1)
X_test1 = pd.concat([X_test_norm,X_test_cats_encoded], axis = 1)
X_test2 = pd.concat([X_test_norm,X_test_cats_encoded], axis = 1)

X_train1.head()


,length,rental_rate,rental_duration,Foreign,Classics,Action,Sports,Children,Animation,Sci-Fi,...,Horror,Games,Drama,Family,Comedy,G,PG-13,NC-17,PG,R
0,0.971223,0.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.374101,1.0,0.75,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.071942,0.0,1.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.424460,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.273381,0.5,0.50,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [181]:
# Import model object for training.
from sklearn.linear_model import LogisticRegression

# Instantiate and train model
LR_Model1 = LogisticRegression(random_state=34, solver='lbfgs',multi_class='multinomial').fit(X_train1, y_train)
Train1_predictions = LR_Model1.predict(X_train1) 
classification_score = LR_Model1.score(X_train1, y_train)
classification_score


0.6405475997478159

In [182]:
# Test model performance
predictions1 = LR_Model1.predict(X_test1)
test_score1 = LR_Model1.score(X_test1, y_test)
test_score1

0.6438327379701618

In [183]:
# Review model performance using confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions1)



array([[3064,    0],
       [1695,    0]], dtype=int64)

In [184]:
pd.Series(predictions1).value_counts()

0    4759
dtype: int64